In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [35]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('../Data/action_log_features.csv')

In [24]:
Q3_2019 = df[(df['at'] >= '2019-07-01') & (df['at'] < '2019-10-01')]
Q4_2019 = df[(df['at'] >= '2019-10-01') & (df['at'] < '2020-01-01')]
Q1_2020 = df[(df['at'] >= '2020-01-01') & (df['at'] < '2020-04-01')]
Q2_2020 = df[(df['at'] >= '2020-04-01') & (df['at'] < '2020-07-01')]
Q3_2020 = df[(df['at'] >= '2020-07-01') & (df['at'] < '2020-10-01')]
Q4_2020 = df[(df['at'] >= '2020-10-01') & (df['at'] < '2021-01-01')]
Q1_2021 = df[(df['at'] >= '2021-01-01') & (df['at'] < '2021-04-01')]
Q2_2021 = df[(df['at'] >= '2021-04-01') & (df['at'] < '2021-07-01')]

In [ ]:
red_light = [
    'add_snippet', 'auth_confirm_caution', 'change_contact_status', 'change_password', 'change_service_user_status',
    'clear_account_settings', 'deactivate_workflow', 'download_task_file', 'download_task_files', 'download_template',
    'export_contacts', 'export_template', 'export_workflow', 'remove_account', 'remove_action',
    'remove_contact', 'remove_draft', 'remove_inform', 'remove_role', 'remove_snippet',
    'remove_status', 'remove_workflow', 'reset_password', 'set_new_pass', 'suspend_account',
    'suspend_role', 'two_factor_email'
]

yellow_light = [
    'account_access', 'add_account', 'add_action', 'add_contact', 'add_draft',
    'add_group', 'add_html_template', 'add_role', 'add_status', 'add_workflow',
    'adm_open_settings', 'edit_account', 'edit_account_uname', 'edit_action', 'edit_contact',
    'edit_docx_template', 'edit_draft', 'edit_group', 'edit_html_template', 'edit_profile',
    'edit_role', 'edit_snippet', 'edit_status', 'edit_workflow', 'edit_workflow_group',
    'resend_token', 'token_login', 'unblock_password_reset', 'unsuspend_account', 'unsuspend_role'
]

green_light = [
    'apply_label', 'attach_task_file', 'attach_template', 'clone_workflow', 'create_task',
    'create_task', 'import_contacts', 'import_workflow', 'login','logout', 'make_available',
    'make_unavailable', 'open_account', 'open_action', 'open_contact', 'open_docx_template',
    'open_group', 'open_html_template', 'open_incident_log', 'open_new_account', 'open_new_action',
    'open_new_contact', 'open_new_docx_template', 'open_new_group', 'open_new_html_template', 'open_new_role',
    'open_new_snippet', 'open_new_status', 'open_new_workflow', 'open_perform_action', 'open_profile',
    'open_role', 'open_snippet', 'open_status', 'open_task', 'open_workflow',
    'open_workflow_group', 'perform_action', 'reset_req', 'set_action_state', 'set_status_state'
]

In [68]:
Q2_2021[Q2_2021.columns[2:]].sum()[Q2_2021[Q2_2021.columns[2:]].sum() == 0].index

Index(['add_group', 'add_html_template', 'add_snippet', 'add_status',
       'add_workflow', 'attach_template', 'change_contact_status',
       'change_service_user_status', 'clone_workflow', 'deactivate_workflow',
       'download_template', 'edit_docx_template', 'edit_html_template',
       'edit_status', 'edit_workflow_group', 'export_contacts',
       'import_contacts', 'import_workflow', 'logout', 'open_docx_template',
       'open_incident_log', 'open_new_docx_template', 'open_new_group',
       'open_new_html_template', 'open_new_snippet', 'open_new_workflow',
       'open_status', 'open_workflow_group', 'remove_contact', 'remove_role',
       'remove_snippet', 'remove_status', 'remove_workflow', 'reset_password',
       'set_action_state', 'set_status_state', 'two_factor_email',
       'unsuspend_role'],
      dtype='object')

In [64]:
Q2_2021[Q2_2021.columns[2:]].sum().sort_values()

unsuspend_role                     0.0
open_workflow_group                0.0
open_status                        0.0
download_template                  0.0
open_new_workflow                  0.0
open_new_snippet                   0.0
open_new_html_template             0.0
edit_docx_template                 0.0
import_contacts                    0.0
open_new_group                     0.0
edit_html_template                 0.0
open_incident_log                  0.0
open_docx_template                 0.0
edit_status                        0.0
edit_workflow_group                0.0
export_contacts                    0.0
logout                             0.0
open_new_docx_template             0.0
clone_workflow                     0.0
deactivate_workflow                0.0
change_service_user_status         0.0
two_factor_email                   0.0
set_status_state                   0.0
set_action_state                   0.0
add_group                          0.0
add_html_template        